# Modeling exercise

## General Instructions

* Submission date: 14.5.2023
* Submission Method: Link to your solution notebook in [this sheet](https://docs.google.com/spreadsheets/d/1GNPESGIhJpPb7LwMAyjF5qpJfZQak_mLkE3i5Y7a_VA/edit?usp=sharing).

In [1]:
import sys; sys.path.append('../src')
import numpy as np
import plotly_express as px

In [2]:
import pandas as pd
import ipywidgets as widgets

In [3]:
from datasets import make_circles_dataframe, make_moons_dataframe

## Fitting and Overfiting 

The goal of the following exercise is to:
* Observe overfitting due to insuffient data
* Observe Overfitting due to overly complex model
* Identify the overfitting point by looking at Train vs Test error dynamic
* Observe how noise levels effect the needed data samples and model capacity

To do so, you'll code an experiment in the first part, and analyze the experiment result in the second part.

### Building an experiment

Code:

1. Create data of size N with noise level of magnitude NL from datasets DS_NAME. 
1. Split it to training and validation data (no need for test set), use 80%-20%. 
1. Use Logistic regression and Choose one complex model of your choice: [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), [SVM with RBF kernel](https://scikit-learn.org/stable/modules/svm.html) with different `gamma` values or [Random forest classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) with differnt number of `min_samples_split`. 
1. Train on the train set for different hyper parameter values. compute:
   1. Classification accuracy on the training set (TRE)
   1. Classification accuracy on the validation set (TESTE)
   1. The difference beteen the two above (E_DIFF)
1. Save DS_NAME, N, NL, CLF_NAME, K, TRE, TESTE, E_DIFF and the regularization/hyper param (K, gamma or min_samples_split and regularization value for the linear regression classifier)

Repeat for:
* DS_NAME in Moons, Circles
* N (number of samples) in [5, 10, 50, 100, 1000, 10000]
* NL (noise level) in [0, 0.1, 0.2, 0.3, 0.4, 0.5]
* For the complex model: 10 Values of hyper parameter of the complex model you've chosen.
* For the linear model: 5 values of ridge (l2) regularization - [0.001, 0.01, 0.1, 1, 10, 100, 1000]

In [39]:
# creating dataframe with data
from sklearn.datasets import make_moons, make_circles
import pandas as pd

final_data_df = pd.DataFrame()

for NL in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
    moons_df = make_moons_dataframe(n_samples=10000, noise_level=NL)
    moons_df['dataset_name']='moons'
    moons_df['noise_level'] = NL
    circles_df = make_circles_dataframe(n_samples=10000, noise_level=NL)
    circles_df['dataset_name']='circles'
    circles_df['noise_level'] = NL
    final_data_df = pd.concat([final_data_df, moons_df, circles_df])
    # final_data_df = final_data_df.append(circles_df)
        
final_data_df.sample(10)

,x,y,label,dataset_name,noise_level
2223,1.317696,0.725208,A,moons,0.2
3855,0.469471,0.626524,A,moons,0.3
7135,-0.637424,0.770513,A,circles,0.0
7508,0.839972,-0.414582,A,circles,0.2
9006,-0.531308,0.598090,B,circles,0.0
349,0.642936,-0.618427,B,circles,0.1
5448,0.033430,1.558600,A,circles,0.4
3627,0.014074,-0.799876,B,circles,0.0
3563,-0.481201,-0.788939,A,circles,0.5
5462,1.011483,-0.403658,A,circles,0.2


In [18]:
# validating that data created properly
final_data_df.groupby(['dataset_name', 'noise_level']).x.count()

dataset_name  noise_level
circles       0.0            10000
              0.1            10000
              0.2            10000
              0.3            10000
              0.4            10000
              0.5            10000
moons         0.0            10000
              0.1            10000
              0.2            10000
              0.3            10000
              0.4            10000
              0.5            10000
Name: x, dtype: int64

In [95]:
# splitting to train and validation set
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

regularization_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
results = pd.DataFrame()
min_samples_split = 10

for ds_type in ['moons','circles']:
    print(f'Working on {ds_type}')
    for nl in [0, 0.1, 0.2, 0.3, 0.4, 0.5]:
        for n in [10, 50, 100, 1000, 10000]:
            query_text = f"dataset_name == '{ds_type}' & noise_level == {nl}" 
            ds = final_data_df.query(query_text).head(n)
            train_set, validation_set = train_test_split(ds,test_size = 0.2,random_state=3)
            x_train = train_set[['x','y']]
            y_train =train_set['label']
            x_test = validation_set[['x','y']]
            y_test = validation_set['label']
            # logistic regression
            for regularization_value in regularization_values:
                log_reg_fit = LogisticRegression(fit_intercept=True, penalty='l2',C = regularization_value).fit(x_train, y_train)
                train_accuracy = accuracy_score(log_reg_fit.predict(x_train),y_train)
                test_accuracy  = accuracy_score(log_reg_fit.predict(x_test),y_test)
                results = results.append( {"DS_NAME":ds_type, "N":n, "NL":nl, "CLF_NAME":"Logistic Regression", "TRE":train_accuracy, "TESTE":test_accuracy, "E_DIFF":abs(train_accuracy-test_accuracy) ,"REG":regularization_value},ignore_index=True)
            # # random forest
            for min_sample in range(2,min_samples_split):
                random_forest_fit = RandomForestClassifier(min_samples_split=min_sample).fit(x_train, y_train)
                train_accuracy = accuracy_score(random_forest_fit.predict(x_train),y_train)
                test_accuracy  = accuracy_score(random_forest_fit.predict(x_test),y_test)
                results = results.append( {"DS_NAME":ds_type, "N":n, "NL":nl, "CLF_NAME":"Random Forest", "TRE":train_accuracy, "TESTE":test_accuracy, "E_DIFF":abs(train_accuracy-test_accuracy) ,"REG":regularization_value},ignore_index=True)
           
print(results.sample(5))

Working on moons
Working on circles
     DS_NAME      N   NL             CLF_NAME     TRE  TESTE  E_DIFF     REG
771  circles     50  0.4  Logistic Regression  0.5750  0.500  0.0750  1000.0
596  circles  10000  0.1        Random Forest  0.9675  0.836  0.1315  1000.0
283    moons   1000  0.3        Random Forest  0.9750  0.895  0.0800  1000.0
453  circles     10  0.0  Logistic Regression  0.8750  0.500  0.3750     1.0
4      moons     10  0.0  Logistic Regression  1.0000  1.000  0.0000    10.0


### Analysing the expermient results

1. For SVM only, For dataset of size 10k and for each dataset, What are the best model params? How stable is it? 
1. For SVM only, For dataset of size 10k and for each dataset, What is the most stable model and model params? How good is it in comparison to other models? Explain using bias and variance terminoligy.
1. Does regularization help for linear models? consider different datasets sizes. 
1. For a given noise level of your chioce, How does the train, test and difference error changes with increasing data sizes? (answer for svm and LR seperatly)
1. For a given noise level of your chioce, How does the train, test and difference error changes with increasing model complexity? (answer for svm and LR seperatly)
1. Are the noise level effect the number of datapoints needed to reach optimal test results? 

Bonus:

* For SVM: Select one dataset and with 0.2 noise level. Identify the optimal model params, and visualize the decision boundry learned. 
  * Hint: Use a grid. See classification models notebook 

Answers:

3. Does regularization help for linear models? consider different datasets sizes.

In [112]:
reg = results[results['CLF_NAME'] == "Logistic Regression"]
reg.head(1)
reg.groupby(['REG','N']).mean()

# regularization does help when values are low (up to 10.0) it is helping to increase test accuracy
# and also resuces difference between test and train accuracy

NL       TRE     TESTE    E_DIFF
REG      N                                        
0.001    10     0.25  0.729167  0.375000  0.416667
         50     0.25  0.577083  0.500000  0.122917
         100    0.25  0.561458  0.445833  0.146875
         1000   0.25  0.651042  0.633750  0.030417
         10000  0.25  0.658979  0.651458  0.010958
0.010    10     0.25  0.729167  0.375000  0.416667
         50     0.25  0.625000  0.550000  0.120833
         100    0.25  0.679167  0.570833  0.129167
         1000   0.25  0.656979  0.656667  0.027604
         10000  0.25  0.666094  0.660583  0.010365
0.100    10     0.25  0.729167  0.375000  0.416667
         50     0.25  0.700000  0.633333  0.125000
         100    0.25  0.700000  0.595833  0.137500
         1000   0.25  0.671146  0.677917  0.023021
         10000  0.25  0.671521  0.666583  0.010417
1.000    10     0.25  0.822917  0.666667  0.281250
         50     0.25  0.735417  0.683333  0.102083
         100    0.25  0.725000  0.620833  0.116667
         1000   0.25  0.677500  0.683333  0.021667
         10000  0.25  0.672490  0.667542  0.010260
10.000   10     0.25  0.854167  0.625000  0.270833
         50     0.25  0.747917  0.691667  0.077083
         100    0.25  0.736458  0.629167  0.121875
         1000   0.25  0.678333  0.682917  0.021458
         10000  0.25  0.672583  0.667375  0.010375
100.000  10     0.25  0.875000  0.708333  0.208333
         50     0.25  0.754167  0.691667  0.083333
         100    0.25  0.737500  0.633333  0.118750
         1000   0.25  0.678333  0.682917  0.021458
         10000  0.25  0.672604  0.667417  0.010396
1000.000 10     0.25  0.875000  0.708333  0.208333
         50     0.25  0.754167  0.691667  0.083333
         100    0.25  0.737500  0.633333  0.118750
         1000   0.25  0.678333  0.683333  0.021042
         10000  0.25  0.672615  0.667375  0.010365

4. For a given noise level of your chioce, How does the train, test and difference error changes with increasing model complexity? (answer for svm and LR seperatly)

In [120]:
reg = results[(results['NL'] == 0.1) & (results['CLF_NAME'] == "Logistic Regression")]
reg.head(1)
reg.groupby(['N']).mean()
# train drops a bit but test increases, difference is lower 

,NL,TRE,TESTE,E_DIFF,REG
N,,,,,
10,0.1,0.741071,0.321429,0.419643,158.730143
50,0.1,0.726786,0.635714,0.091071,158.730143
100,0.1,0.716071,0.625000,0.117857,158.730143
1000,0.1,0.682768,0.656786,0.025982,158.730143
10000,0.1,0.693018,0.687821,0.005982,158.730143


5. Are the noise level effect the number of datapoints needed to reach optimal test results?

In [135]:
results.groupby("NL")

,N,TRE,TESTE,E_DIFF,REG
NL,,,,,
0.0,10000,1.0,1.0,0.750,1000.0
0.1,10000,1.0,1.0,1.000,1000.0
0.2,10000,1.0,1.0,0.625,1000.0
0.3,10000,1.0,1.0,0.875,1000.0
0.4,10000,1.0,1.0,0.600,1000.0
0.5,10000,1.0,1.0,0.500,1000.0


## Tips and Hints

For buliding the experiment:

* Start with one dataframe holding all the data for both datastes with different noise level. Use the `make_<dataset_name>_dataframe()` functions below, and add two columns, dataset_name and noise_level, before appending the new dataset to the rest of the datasets. Use `df = pd.DataFrame()` to start with an empty dataframe and using a loop, add data to it using `df = df.append(<the needed df here>)`. Verify that you have 10k samples for each dataset type and noise level by a proper `.value_counts()`. You can modify the 
* When you'll need an N samples data with a specific noise level, use `query()` and `head(n)` to get the needed dataset. 
* Use sklearn `train_test_split()` method to split the data with `test_size` and `random_state` parameters set correctly to ensure you are always splitting the data the same why for a given fold `k`. Read [the docs](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) if needed. 
* You can also not create your own data splitter, and instead use `model_selection.cross_validate()` from sklearn. You'll need to ask for the train erros as well as the test errors, see [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html).
* Use prints in proper location to ensure the progress of the experiment. 

**If you get stuck, and need refernce, scroll to the end of the notebook to see more hints!**

## Moons dataset

In [12]:
from sklearn.datasets import make_moons

In [13]:
moons_df = make_moons_dataframe(n_samples=1000, noise_level=0.1)
moons_df.head()

,x,y,label
0,0.552235,0.831124,A
1,0.828307,-0.527616,B
2,1.721499,-0.049019,B
3,0.450451,0.827117,A
4,0.099954,0.961871,A


In [14]:
@widgets.interact
def plot_noisy_moons(noise_level = widgets.FloatSlider(value=0, min=0, max=0.5, step=0.05)):
    moons_df = make_moons_dataframe(n_samples=1000, noise_level=noise_level)
    return px.scatter(moons_df, x='x', y='y', color = 'label')

interactive(children=(FloatSlider(value=0.0, description='noise_level', max=0.5, step=0.05), Output()), _dom_c…

## Circles Dataset

In [15]:
circles_df = make_circles_dataframe(n_samples=500, noise_level=0)
circles_df.head()

,x,y,label
0,-0.070281,0.796907,B
1,-0.701045,-0.385403,B
2,0.763892,0.237633,B
3,0.332820,-0.942991,A
4,-0.691139,0.402899,B


In [16]:
@widgets.interact
def plot_noisy_circles(noise_level = widgets.FloatSlider(value=0, min=0, max=0.5, step=0.05)):
    df = make_circles_dataframe(1000, noise_level)
    return px.scatter(df, x='x', y='y', color = 'label')

interactive(children=(FloatSlider(value=0.0, description='noise_level', max=0.5, step=0.05), Output()), _dom_c…

## Appendix

### More hints!

If you'll build the datasets dataframe correctly, you'll have **one** dataframe that has dataset_name and noise_level colmuns, as well as the regular x,y,label colmns. To unsure you've appended everything correctly, groupby the proper colmuns and look at the size:

In [17]:
# Use proper groupby statement to ensure the datasets dataframe contains data as expected. You should see the following result:

Your 

You experiment code should look something like that:

In [19]:
datasets_type = ['circles', 'moons']
k_folds = 10
n_samples = [10, 50, 100, 1000, 10000]
noise_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
clf_types = ['log_reg', 'svm']
hp_range = <'Your hyper parameters ranges here'>
regularization_values = <'Your regularization values here'>
results = []
for ds_type in datasets_type:
    print(f'Working on {ds_type}')
    for nl in noise_levels:
        for n in n_samples:
            ds = datasets.query(<'your query here'>).head(n)
            print(f'Starting {k_folds}-fold cross validation for {ds_type} datasets with {n} samples and noise level {nl}. Going to train {clf_types} classifiers.')
            for k in range(k_folds):
                X, Y = <'Your code here'>
                x_train,x_test,y_train,y_test= <'Your code here'>
                for clf_type in clf_types:
                    if clf_type == 'log_reg':
                        for regularization_value in regularization_values:
                            train_acc, test_acc = <'Your code here'>
                            results.append(<'Your code here'>)
                    if clf_type == 'svm':
                        for gamma in hp_range:
                            train_acc, test_acc = <'Your code here'>
                            results.append(<'Your code here'>)